In [6]:
import pandas as pd
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import operator
import numpy as np

/Users/yishh/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [7]:
G = nx.read_edgelist("../code/0_cleaning/max_wcc_graph.edgelist", create_using=nx.DiGraph(), delimiter="\t", data=True)

In [11]:
post_2005_edges = pd.read_csv("../code/0_cleaning/max_wcc_graph.edgelist", low_memory=False)

In [82]:
total_periods = 132
rng = pd.date_range('1/1/2005', periods= total_periods, freq='M')
months_per_period = 3
for i in range(1,total_periods/months_per_period):
    period = i * months_per_period
    curr_rng = rng[:period]
    curr_start = curr_rng[0]
    curr_end = curr_rng[-1]
#     print curr_rng 
#     print curr_start, curr_end
    
    # post_2005_edges 
    mask = (post_2005_edges['invest_date'] >= curr_start) & (post_2005_edges['invest_date'] <= curr_end)
    period_snapshot = post_2005_edges[mask]
    file_name = "cumulative_snapshots/enddate_" + str(curr_end)[:10] + ".csv"
#     print file_name
    period_snapshot.to_csv(file_name, index='false')
#     if i ==3:
#         break


NameError: name 'post_2005_edges' is not defined

In [ ]:
total_periods = 132
rng = pd.date_range('1/1/2005', periods= total_periods, freq='M')
months_per_period = 3
for i in range(1,total_periods/months_per_period):
    period = i * months_per_period
    curr_rng = rng[:period]
    curr_start = curr_rng[0]
    curr_end = curr_rng[-1]
#     print curr_rng 
#     print curr_start, curr_end
    
    # post_2005_edges 
    mask = (post_2005_edges['invest_date'] >= curr_start) & (post_2005_edges['funded_at'] <= curr_end)
    period_snapshot = post_2005_edges[mask]
    file_name = "cumulative_snapshots/enddate_" + str(curr_end)[:10] + ".csv"
#     print file_name
    period_snapshot.to_csv(file_name, index='false')
#     if i ==3:
#         break


In [85]:
#calculate all shortest path lengths
# G = nx.path_graph(5)
all_path_lengths = nx.all_pairs_shortest_path_length(G)
all_path_lengths_sorted = []
for i in range(len(all_path_lengths)):
    p = all_path_lengths[i]
    curr_distances =  sorted(list(zip(p.keys(),p.values())), key = lambda x: x[1], reverse = True)
    all_path_lengths_sorted.append(curr_distances)
    
all_path_lengths_sorted

[[(4, 4), (3, 3), (2, 2), (1, 1), (0, 0)],
 [(4, 3), (3, 2), (0, 1), (2, 1), (1, 0)],
 [(0, 2), (4, 2), (1, 1), (3, 1), (2, 0)],
 [(0, 3), (1, 2), (2, 1), (4, 1), (3, 0)],
 [(0, 4), (1, 3), (2, 2), (3, 1), (4, 0)]]

In [86]:
#calculate number of paths between every other node
# G = nx.path_graph(5)
g_size = len(G)

all_number_paths = np.zeros((g_size,g_size))

for i in range(g_size):
    for j in range(i,g_size):
        if i == j:
            continue
        all_paths = nx.all_simple_paths(G, source=i, target=j)
        num = len(list(all_paths))
        all_number_paths[i,j] = num
        all_number_paths[j,i] = num

all_number_paths


array([[ 0.,  1.,  1.,  1.,  1.],
       [ 1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.],
       [ 1.,  1.,  1.,  1.,  0.]])

In [89]:
#calculate number of paths between every other node
# G = nx.path_graph(5)
g_size = len(G)

all_number_paths = np.zeros((g_size,g_size))

adamic_index = nx.adamic_adar_index(G)
adamic_index_list = [p for p in adamic_index]
adamic_index_list


[(0, 2, 1.4426950408889634),
 (0, 3, 0),
 (0, 4, 0),
 (1, 3, 1.4426950408889634),
 (1, 4, 0),
 (2, 4, 1.4426950408889634)]

In [91]:
#once i have generated every graph
#for each snapshot
    #generate a graph
    #for each graph
        #get all shortest paths lengths
        #get number paths between every node
        #get adamic index
        #store it properly
        
#random link predictor


In [8]:
G

In [ ]:
post_2005_edges

In [38]:
dates = nx.get_edge_attributes(G, 'date')
weights = nx.get_edge_attributes(G, 'weight')

flat_table = map(lambda x: [x[0], x[1], dates[x]] , dates, weights)
wcc_df = pd.DataFrame(flat_table)
wcc_df.columns = ["investor", "company", "invest_date","weight"]
wcc_df

TypeError: <lambda>() takes exactly 1 argument (2 given)

In [29]:
wcc_df

,company,investor,invest_date
78047,/organization/rsj-private-equity,/organization/cognitive-security,2011-04-25
78048,/organization/rsj-private-equity,/organization/beepl,2011-07-20
78049,/person/kevin-lin,/organization/the-ticket-fairy,2015-05-20
78050,/organization/petra-partners,/organization/new-century-hospice,2010-10-13
78051,/person/emilios-markou,/organization/park-around,2013-10-07
78052,/person/dave-leyrer,/organization/tapiture,2014-02-11
78053,/person/dave-leyrer,/organization/omaze,2015-07-15
78054,/person/oren-etzioni,/organization/algorithmia,2014-08-18
78055,/organization/beamonte-investments,/organization/arthena,2014-12-29
78056,/organization/beamonte-investments,/organization/black-house,2013-09-23
